In [2]:
import torch
import pandas as pd
import numpy as np
import pickle
import torch.utils.data as Data
import import_ipynb
from LSTM import *
import sys
sys.path.append('..')
from helpers import *

In [3]:
vocab_dim = 100
maxlen = 20

embedding_weights = np.load("Word2VecArray.npy") 
# Set a zero vector for words that do not appear in the vocabulary
embedding_weights = np.r_[np.zeros((1, vocab_dim)),embedding_weights]

f = open("../Glove/vocab.pkl", 'rb') 
index_dict = pickle.load(f)    # index dictionary {'word': idx}

# Index each word + 1 because of the zero vector
for key, value in index_dict.items():  
    index_dict[key] = value + 1 

data = []
with open("../twitter-datasets/test_data.txt", "r", encoding='UTF-8') as f:
    for line in f:
        line = line.replace("\t", "").replace("<user>", "").replace("\n", "").replace("<url>", "")
        data.append(line)
        
from torch.nn.utils.rnn import pad_sequence
data = text_to_index_array(index_dict, data)
data = pad_sequence([torch.from_numpy(np.array(x)) for x in data],batch_first=True).float() 
data = text_cut_to_same_long(data,maxlen)
data = creat_wordvec_tensor(embedding_weights,data,100,20)

print("data shape： ", data.shape)

data shape：  (10000, 20, 100)


In [5]:
model = lstm()
model = torch.load('./W2V_LSTM.pt',map_location=torch.device('cpu'))
data = torch.as_tensor(data, dtype=torch.float32)
output,_ = model(data)
pre=torch.argmax(output, dim=1)

In [6]:
pre_list = pre.numpy().tolist ()
idx = [i for i in range(1,len(pre)+1)]
dict_ = {
    "Id" : idx,
    "Prediction" : pre_list
}
pred_df = pd.DataFrame(dict_)
pred_df['Prediction'][pred_df['Prediction'] == 0] = -1
pred_df.to_csv("pred.csv",index = False)